# Non-linear latent variable models

Non-linear latent variable models are a class of statistical models that involve a set of latent (unobserved) variables that are related to observed variables through a non-linear relationship. These models are used to work with complex data sets. Some examples of non-linear latent variable models are autoencoders, variational autoencoders, and generative adversarial networks.

Linear latent variable models and non-linear latent variable models are both types of statistical models that involve a set of latent variables. However, linear latent variable models assume that the relationship between the latent variables and the observed variables is linear. On the other hand, non-linear latent variable models assume that the relationship between the latent variables and the observed variables is non-linear. This means that the relationship cannot be represented by a linear equation, but by a more complex mathematical function, such as polynomials or neural networks.

Apply VAE on a computer vision dataset

Apply VAE on a molecular dataset